<a href="https://colab.research.google.com/github/shardulburde/Exotic-Options-Pricing/blob/main/InstallmentOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [371]:
#Reading Excel Data
#Excel sheet link in next comment
#https://drive.google.com/file/d/1UXq6kIWAaGUmzk41B4ReCg5L_k_vBtas/view?usp=sharing

import pandas as pd
xls = pd.ExcelFile('/content/drive/MyDrive/IndusInd/Exotic Options.xlsx')
df1 = pd.read_excel(xls, 'MktData')
df=pd.read_excel(xls, 'INR_DF')

In [372]:
df

,Tenor,DCF,Discount
0,Spot,0.000000,1.000000
1,1M,0.083333,0.995518
2,2M,0.166667,0.991318
3,3M,0.250000,0.986878
4,6M,0.500000,0.974541
5,1Y,1.000000,0.949845
6,2Y,2.000000,0.903240
7,3Y,3.000000,0.853836
8,4Y,4.000000,0.806388
9,5Y,5.000000,0.756827


In [373]:
#Interpolating MIFOR Discount Factors
from scipy import interpolate

def disc_fac(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(13):
    x=pd.DataFrame([df._get_value(i+1,1,True)])
    y=pd.DataFrame([df._get_value(i+1,2,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return float(interpolate.splev(tenor, tck))

disc_fac(0)

0.9989339856526179

In [388]:
import numpy as np

def discount(factors,i):
  value=factors[i+1]/factors[i]
  return value



In [375]:
#Interpolating Market Vol Data
from scipy import interpolate

def atm(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,4,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

def rr25d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,7,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

def fly25d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,10,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)


def rr10d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,13,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)

  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)
      

def fly10d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,16,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
    
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

In [376]:
#Making Call Delta functions that return vol for given tenor from mkt data

def d10(tenor):
  vol=atm(tenor)+fly10d(tenor)+rr10d(tenor)/2

  return vol

def d25(tenor):
  vol=atm(tenor)+fly25d(tenor)+rr25d(tenor)/2

  return vol

def d75(tenor):
  vol=atm(tenor)+fly25d(tenor)-rr25d(tenor)/2

  return vol

def d90(tenor):
  vol=atm(tenor)+fly10d(tenor)-rr10d(tenor)/2

  return vol

In [377]:
#Interpolation for vol for tenor b/w 0.1, 0.25, 0.5, 0.75 and 0.9 call delta

from scipy import interpolate

def vol(delta,tenor):

  y1=d10(tenor)
  y2=d25(tenor)
  y3=atm(tenor)
  y4=d75(tenor)
  y5=d90(tenor)

  x_points = [0.1, 0.25, 0.5, 0.75, 0.9]
  y_points = [y1,y2,y3,y4,y5]

  tck = interpolate.splrep(x_points, y_points)
  return float(interpolate.splev(delta, tck))

vol(0.1,1)


0.08452500000000002

In [378]:
#Making Vol Lookup Function

import numpy as np
import math
import scipy
from scipy.stats import norm


def sigma(strike,spot,tenor,rate):
  
  v = atm(tenor)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-v*v/2)*tenor)/(v*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    v=vol(delta,tenor) 

  return v

sigma(80,74,2,0.05)


0.05694212636585169

In [379]:
import numpy as np
def v(p1,strike,fwd,tenor,rate,n,i,factors):

  V=np.empty([1,n+1],dtype=float) 
  V[0,n]=max(0,fwd-strike)
  for k in range(n):
    l=n-1-k
    V[0,l]=max(0,discount(factors,l)*V[0,l+1]-p1)

  value=float(V[0,i])
  return value 

def g1(p1,strike,fwd,tenor,rate,n,p0,factors):
  value=v(p1,strike,fwd,tenor,rate,n,1,factors)-p0
  return value

def g2(p1,strike,fwd,tenor,rate,n,factors):
  value=v(p1,strike,fwd,tenor,rate,n,1,factors)-p1
  return value




In [380]:
import math
import numpy as np



def f1(strike, spot, tenor, rate, volatility, t, n, numsim, p1,factors):
  
  volatility=volatility/100
  
  if volatility==0:
    volatility=sigma(strike,spot,tenor,rate)
   
  sum = 0.0
  it=round(tenor*365/t)
  time=np.linspace(0.0,tenor,num=(n+1))
  

  for l in range(numsim):

    fwd=spot
    
    for j in range(it):
      fwd = fwd * math.exp((rate - (volatility * volatility) / 2) * (t/365) + volatility * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
  
    p0 = v(p1,strike,fwd,tenor,rate,n,1)*discount(factors,0)
    sum = sum + p0
    
  avg_p0=(sum/numsim)
 
  return avg_p0


In [381]:
import math
import numpy as np
import scipy.optimize as opt

def f2(strike, spot, tenor, rate, volatility, t, n, numsim, p0,factors):

  volatility=volatility/100

  if volatility==0:
    volatility=sigma(strike,spot,tenor,rate)
 

  sum = 0.0
  it=round(tenor*365/t)
  time=np.linspace(0.0,tenor, num=(n+1))
  
  p1=1
  for l in range(numsim):
    
    fwd=spot
    
    for j in range(it):
      fwd = fwd * math.exp((rate - (volatility * volatility) / 2) * (t/365) + volatility * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
 
    
    args=(strike,fwd,tenor,rate,n,p0,factors)
    p1=float(opt.fsolve( g1, p1, args))
    sum = sum + p1
    

  avg_p1=(sum/numsim)
 
  return avg_p1


In [382]:
import math
import numpy as np
import scipy.optimize as opt

def f3(strike, spot, tenor, rate, volatility, t, n, numsim,factors):

  volatility=volatility/100

  if volatility==0:
    volatility=sigma(strike,spot,tenor,rate)
  

  sum = 0.0
  Sum=0.0
  it=round(tenor*365/t)
  time=np.linspace(0.0,tenor, num=(n+1))
  
  p1=1
  for l in range(numsim):
    
    fwd=spot
    
    for j in range(it):
      fwd = fwd * math.exp((rate - (volatility * volatility) / 2) * (t/365) + volatility * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))
 
    BS=math.exp(-rate*tenor)*max(0,fwd-strike)/n
    args=(strike,fwd,tenor,rate,n,factors)
    p1=float(opt.fsolve( g2, p1, args))
    sum = sum + p1
    Sum=Sum+BS

  avg_p1=(sum/numsim)
  avg_BS=(Sum/numsim)
  return avg_BS,avg_p1


In [383]:
def function(strike,spot,tenor,rate,volatility,t,n,numsim,Option,factors):

  if Option==1:
    p1=float(input('Enter future constant installments(p1):'))
    value1=f1(strike, spot, tenor, rate, volatility, t, n, numsim, p1,factors)
    print('Input(p1):',p1,',Output(p0):',value1)

  if Option==2:
    p0=float(input('Enter initial amount(p0):'))
    value2=f2(strike, spot, tenor, rate, volatility, t, n, numsim, p0,factors)
    print('Input(p0):',p0,',Output(p1):',value2)

  if Option==3:
    value3=f3(strike, spot, tenor, rate, volatility, t, n, numsim,factors)
    print('p_BS:',value3[0],',p0=p1=',value3[1])

Input Data
 down below, run it to check apt Forward


In [384]:
from datetime import date
import math

#(YYYY, MM, DD)
start_date = date(2021, 7, 5)
end_date = date(2022, 7, 5)
num_days=(end_date-start_date).days
Strike=75.5
Spot=74.5
Tenor=num_days/365
INRr= 4.535             #Enter INR Rate in %
USDr=  0.05             #Enter USD Rate in %
Rate=(INRr-USDr)/100           
Volatility=0     #Enter 0 if vol lookup is to be performed #Enter vol in %
DelT=1
n=4
Numsim=1000



time=np.linspace(0.0,tenor,num=n+1)
factors=np.linspace(0.0,tenor,num=n+1)

for k in range(n+1):
  factors[k]=disc_fac(time[k])

Fwd=Spot*math.exp(Rate*Tenor)
print('Fwd=',Fwd)

Fwd= 77.9173870785232


In [385]:
import warnings
warnings.filterwarnings('ignore', 'The iteration is not making good progress')
"""
Enter Respective Option Number

Option 1: I/P: p1(future constant Installments),  O/P: p0(Initial amount to be paid) 
Option 2: I/P: p0(Initial amount to be paid),     O/P: p1(future constant Installments)
Option 3: I/P: N/A                                O/P: p0 such that all payments are same(p0=p1)

"""

Option=3

function(Strike,Spot,Tenor,Rate,Volatility,DelT,n,Numsim,Option,factors)


p_BS: 0.7912994536801189 ,p0=p1= 0.7816468705861954
